# Imports

In [1]:
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
from datetime import datetime
import pandas as pd
from connection_keys import connection, alchemy_connection

### Setup para abrir o navegador com o Selenium e puxar a primeira página

In [2]:
#--| Setup
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--disable-extensions')
#options.add_argument("--headless")
browser = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
#--| Parse or automation
browser.get('https://www.kabum.com.br/hardware/placa-de-video-vga?pagina=1')
html = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(5)

### Configuiração para percorer as páginas e salvar todos dados com a taga referente ao produto em uma lista

In [3]:
pagina = 0
html_produtos = []
flag = 0

while flag == 0:
    items = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.kabum.com.br/hardware/placa-de-video-vga?pagina={pagina}')
            html = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

### Tranformação da lista de listas em uma lista normal para percorer com mais facildiade os atributos do produto

In [4]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

### Percorrendo a lista, capurando e salvando cada atributo no dicionario

In [5]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)


### Transformando o dict em Dataframe e visualizando uma amostra dos dados

In [6]:
df = pd.DataFrame(produtos)
df.sample(10)

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,disponibilidade,data,hora,loja
318,MSI,Placa de Vídeo MSI NVIDIA GeForce RTX 2060 Ven...,0,"2.235,18","1.899,90",Indisponivel,2020-09-11,18:56:58,Kabum
141,Rise Mode,Suporte para Placa de Video Rise Mode Ice Col...,"30,47","29,29","24,90",Disponivel,2020-09-11,18:56:58,Kabum
236,Zotac,Placa de Vídeo Zotac Gaming NVIDIA GeForce GTX...,"1.882,24","1.588,12","1.349,90",Disponivel,2020-09-11,18:56:58,Kabum
377,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,0,"2.023,41","1.719,90",Indisponivel,2020-09-11,18:56:58,Kabum
150,Asus,Placa de Vídeo Asus Dual AMD Radeon RX 5600 XT...,"2.681,06","2.470,47","2.099,90",Disponivel,2020-09-11,18:56:58,Kabum
369,Liketec,"Suporte para Placa de Vídeo Liketec Nexxus 2, ...",0,"23,41","19,90",Indisponivel,2020-09-11,18:56:58,Kabum
158,Evga,Placa de Vídeo EVGA NVIDIA GeForce RTX 2070 Su...,"5.555,18","4.705,76","3.999,90",Disponivel,2020-09-11,18:56:58,Kabum
343,PCYes,"Placa de Vídeo PCYes AMD Radeon R7 240,\t4GB, ...",0,"776,35","659,90",Indisponivel,2020-09-11,18:56:58,Kabum
218,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 105...,"1.364,59","1.246,94","1.059,90",Disponivel,2020-09-11,18:56:58,Kabum
125,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 166...,"2.705,76","2.470,47","2.099,90",Disponivel,2020-09-11,18:56:58,Kabum



### Criando a conexão com o banco e salvando os dados

In [7]:
engine = alchemy_connection('db_hardware')
df.to_sql('graphic_card', con=engine, if_exists='append', index = False)

----------------------------------------------------------------------------------------------------

In [8]:
# containers = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
# https://www.kabum.com.br/cgi-local/site/listagem/listagem.cgi?string=processador&btnG=&pagina=1&ordem=5&limite=30&prime=false&marcas=[%225%22,%226%22]&tipo_produto=[]&filtro=[]

In [9]:
# images = html.findAll('div', {"class":"sc-fznKkj fEpBwY"})

# # images.img['src']
# for image in images:
#     print(image.img['src'])
#     # if "indisponivel" in image.img['src']:
#     #     print('Indisponivel')
#     # elif "disponivel" in image.img['src']:
#     #     print('Disponivel')

In [10]:
# preco_antigo_cartao = float((container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2].replace('.', '').replace(',', '.'))
# preco_atual_cartao =  float((container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1].replace('.', '').replace(',', '.'))
# preco_boleto = float((container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1].replace('.', '').replace(',', '.'))


In [11]:
# print(preco_antigo_cartao)
# # print(re.search('^(([1-9]\d{0,2}(\.\d{3})*)|(([1-9]\.\d*)?\d))(\,\d\d)?', preco_atual_cartao).group())
# print(preco_atual_cartao)
# print(preco_boleto)